## [ Soynlp ] 학습 기반 토크나이저 <hr>

- 품사 태깅, 단어 토큰화 등을 지원하는 단어 토크나이저
- 비지도 학습으로 단어 토큰화 -> 데이터에 자주 등장하는 단어들을 단어로 분석
- 내부적으로 단어 점수 표로 동작

In [2]:
#!pip install soynlp

In [5]:
# 기존 형태소 분석기: 신조어나 형태소 분석기에 등록되지 않은 단어를 제대로 구분하지 못함
from konlpy.tag import Okt

tokenizer = Okt()
print(tokenizer.morphs(phrase='에이비식스 이대휘 1월 최애돌 기부 요정입니다'))

#형태소 분석시 매개변수 stem = True 설정. 
print(tokenizer.morphs(phrase='에이비식스 이대휘 1월 최애돌 기부 요정입니다.', stem=True)) # 어근 찾아 원형 돌려주는 친구
print(tokenizer.morphs(phrase='에이비식스 이대휘 1월 최애돌 기부 요정입니다', norm=True))# 정규화시켜줌

['에이', '비식스', '이대', '휘', '1월', '최애', '돌', '기부', '요정', '입니다']
['에이', '비식스', '이대', '휘', '1월', '최애', '돌', '기부', '요정', '이다', '.']
['에이', '비식스', '이대', '휘', '1월', '최애', '돌', '기부', '요정', '입니다']


[ Sonlpy ] 사용 : 말뭉치(Corpus) 데이터셋으로 학습 진행 후 사용 가능

In [6]:
filename = '../datas/text_data.txt'

In [7]:
from soynlp import DoublespaceLineCorpus #하나로 통합된 문서 데이터 분리하기
from soynlp.word import WordExtractor   # 단어 추출기


In [8]:
# 훈련 데이터 문서 분리하기
corpus = DoublespaceLineCorpus(corpus_fname=filename)
print(f'훈련 데이터 문서 : {len(corpus)}개')

훈련 데이터 문서 : 30091개


In [10]:
#soynlpy 학습 진행하기
word_extractor = WordExtractor()

# 학습 진행하며 단어별 점수
word_extractor.train(sents=corpus)

# 단어별 점수표 추출하기
word_scores = word_extractor.extract()

training was done. used memory 1.029 Gb
all cohesion probabilities was computed. # words = 223348
all branching entropies was computed # words = 361598
all accessor variety was computed # words = 361598


In [11]:
# 단어별 점수표 확인하기
for idx, key in enumerate(iterable= word_scores.keys()):
    print(f'[{idx}] = {key}')
    if idx == 30:
        break


[0] = 필
[1] = 룽
[2] = 게
[3] = 딜
[4] = 잔
[5] = 새
[6] = 첸
[7] = 컥
[8] = 궂
[9] = 텅
[10] = 뵙
[11] = 너
[12] = 줴
[13] = 띠
[14] = ㄱ
[15] = 덥
[16] = 팍
[17] = 노
[18] = 줍
[19] = 획
[20] = 앵
[21] = 얇
[22] = 뮐
[23] = 텃
[24] = 인
[25] = 칵
[26] = 횃
[27] = 순
[28] = 진
[29] = 갉
[30] = 땀


응집 확률: 내부 문자열(Substring)이 얼마나 자주 응집하여 나타나는지를 판단하는 척도
- 원리 : 문자열을 문자 단위로 분리, 왼쪽부터 순서대로 문자를 추가
        각 문자열이 주어졌을 때 그 다음 문자가 나올 확률을 계산 / 누적곱한 값
- 값이 높을수록 전체 말뭉치(corpus)에서 이 문자열 시퀀스는 하나의 단어로 등장할 가능성이 높다. 

In [12]:
word_scores['바다'].cohesion_forward 

0.06393648140409527

In [13]:
word_scores['바다가'].cohesion_forward

0.05007307656055328

In [14]:
word_scores['바다에'].cohesion_forward

0.11518621707955429

## L Tokenizer <hr>
- 띄어쓰기 단위로 나눈 어절 토큰 : L토큰, R 토큰 (공원에 = 공원 + 에, 공부하러 = 공부 + 하러)
- 분리 기준 : 점수가 가장 높은 L토큰을 찾아내는 원리

In [17]:
from soynlp.tokenizer import LTokenizer

scores = {word:score.cohesion_forward for word, score in word_scores.items()}

l_tokenizer = LTokenizer(scores=scores)
l_tokenizer.tokenize('국제사회와 우리의 노력들로 범죄를 척결하자', flatten=False)

[('국제사회', '와'), ('우리', '의'), ('노력', '들로'), ('범죄', '를'), ('척결', '하자')]

### 최대 점수 토크나이저
- 띄어쓰기가 되지 않는 문장에서 점수가 높은 글자 시퀀스를 순차적으로 찾아내는 토크나이저
- 띄어쓰기가 되어있지 않은 문장을 넣어 점수를 통해 토큰화된 결과


In [18]:
from soynlp.tokenizer import MaxScoreTokenizer

maxscore_tokenizer = MaxScoreTokenizer(scores=scores)
maxscore_tokenizer.tokenize('국제사회와우리의노력들로범죄를척결하자')


['국제사회', '와', '우리', '의', '노력', '들로', '범죄', '를', '척결', '하자']

### SOyNlp를 이용한 반복되는 문자 정제하기
- ㅠㅠ, ㅋㅋ, ㅎㅎ등의 이모티콘의 경우 불필요하게 연속되는 경우 많음
- ㅋㅋ, ㅋㅋㅋ, ㅋ, ㅋㅋㅋㅋ 등의 경우를 모두 서로 다른 단어로 처리하는 것은 불필요
--> 반복되는 문자는 하나로 정규화

In [20]:
from soynlp.normalizer import *
import string

print(string.punctuation)
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋㅋ 이영화존잼쓰ㅜㅜㅜㅜ', num_repeats=1))
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋㅋㅋㅋㅋ 이영화존잼쓰ㅜㅜㅜㅜㅜ', num_repeats=2))
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 이영화존잼쓰ㅜㅜㅜㅜㅜㅜㅜㅜ', num_repeats=2))

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
아ㅋ 이영화존잼쓰ㅜ
아ㅋㅋ 이영화존잼쓰ㅜㅜ
아ㅋㅋ 이영화존잼쓰ㅜㅜ


In [23]:
#pip install customized_konlpy

In [25]:
from ckonlpy.tag import Twitter

In [26]:
twt = Twitter()
twt.morphs('은경이는 사무실로 갔습니다.')

c:\Users\kdp\anaconda3\envs\NLP_38\lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


['은', '경이', '는', '사무실', '로', '갔습니다', '.']

In [28]:
# 형태소 분석기에 사전 추가하기
twt.add_dictionary(words='은경이', tag='Noun')

In [30]:
twt.morphs('은경이는 사무실로 갔습니다.')

['은경이', '는', '사무실', '로', '갔습니다', '.']

In [31]:
twt.pos('은경이는 사무실로 갔습니다.')

[('은경이', 'Noun'),
 ('는', 'Josa'),
 ('사무실', 'Noun'),
 ('로', 'Josa'),
 ('갔습니다', 'Verb'),
 ('.', 'Punctuation')]